In [1]:
import os
import cv2
import numpy as np
import json
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DATA_PATH = os.path.join(os.getcwd(), '../data/straight_dense')
print(DATA_PATH)
PROCESSED_DATA_PATH = os.path.join(os.getcwd(), '../processed/straight_dense')
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)
if not os.path.exists(os.path.join(PROCESSED_DATA_PATH,'images')):
    os.makedirs(os.path.join(PROCESSED_DATA_PATH,'images'))

/home/boomalope/nerf_estimator/src/nerf_estimator/nerf_vision/scripts/../data/straight_dense


In [3]:
#Gazebo Param: 
#Definition: https://github.com/ros-simulation/gazebo_ros_pkgs/blob/398cde933cba5d72921055d57d4faa47c4445654/gazebo_plugins/src/gazebo_ros_camera_utils.cpp#L524

In [4]:
#Camera parameters
#Definitions: https://docs.nerf.studio/quickstart/data_conventions.html

fl_x = 762.72 # focal length in x
fl_y = 762.72# focal length in y
w = 1280 # image width
h = 720 # image height
cx = w//2 # optical center in x
cy = h//2 # optical center in y
camera_model = "OPENCV" # camera model
k1 = 0 # radial distortion coefficient k1
k2 = 0 # radial distortion coefficient k2
k3 = 0 # radial distortion coefficient k3
p1 = 0 # tangential distortion coefficient p1
p2 = 0 # tangential distortion coefficient p2

In [5]:
def from_tf_to_matrix(tf):
    #tf is the frame of the robot relative to the camera
    xyz = tf[:3]
    q = tf[3:7]
    goal = tf[7:]

    #Rotation matrix
    r = Rotation.from_quat(q)
    r = r.as_matrix()

    #Translation matrix
    t = np.zeros((4,4))
    t[:3,:3] = r.T
    t[:3,3] = -np.dot(r.T, xyz)
    t[3,3] = 1

    return t

In [6]:
data_cnt = 0
frames = []
while True:
    #Get data paths
    data_img = os.path.join(DATA_PATH, '{}.png'.format(data_cnt))
    data_tf= os.path.join(DATA_PATH, '{}.txt'.format(data_cnt))

    if not os.path.exists(data_img):
        break
    
    #Get tf as np array
    tf = np.loadtxt(data_tf, delimiter=',')


    file_path = "images/{}.png".format(data_cnt)
    transform_matrix = from_tf_to_matrix(tf)

    #Append to frames
    frames.append({
        "file_path": file_path,
        "transform_matrix": transform_matrix.tolist(),
    })

    print(transform_matrix[0:3,3])

    #Save image in processed folder
    img = cv2.imread(data_img)
    cv2.imwrite(os.path.join(PROCESSED_DATA_PATH, file_path), img)
    
    data_cnt += 1     
    if data_cnt == 400:
        break   

[-1.01731874 -0.67357109  2.00017261]
[-1.04951054 -0.62257786  2.00022266]
[-1.19532308 -0.2477937   1.99998266]
[-1.20740874  0.18359116  1.99980815]
[-1.10850079  0.51127357  1.99974929]
[-0.87131572  0.85641968  1.99936915]
[-0.62043746  1.05232454  1.99947085]


[-0.243317    1.19713299  1.99923093]
[0.13613304 1.21419064 1.99914405]
[0.49386442 1.11759755 1.99920795]
[0.83653582 0.88994941 1.99933388]
[1.04927883 0.62455257 1.99948855]
[1.20499241 0.20202649 1.99946013]
[ 1.21392226 -0.1318721   1.99973276]
[ 1.10232968 -0.52670574  1.99981664]
[ 0.88698576 -0.83880271  2.00001642]
[ 0.60330311 -1.06103032  2.00015274]
[ 0.23467833 -1.19777275  2.00041786]
[-0.19027466 -1.20559037  2.00041375]
[-0.50985803 -1.10871688  2.00042376]
[-1.64666688 -1.12230528  2.00034222]
[-1.76704687 -0.92184975  2.00030701]
[-1.95884192 -0.37067403  2.00011952]
[-1.98472174  0.19537472  1.99987526]
[-1.79065206  0.87944216  1.99951327]
[-1.48391484  1.33431686  1.99929349]
[-0.94656217  1.75697525  1.99887934]
[-0.42343717  1.95058587  1.99878516]
[0.23964139 1.98187719 1.99848053]
[0.79787158 1.82944581 1.99882386]
[1.43064752 1.39161374 1.99892923]
[1.72094607 1.00948994 1.99903875]
[1.96245268 0.35805234 1.99939971]
[ 1.98076834 -0.22866499  1.99981921]
[ 1.

In [7]:
transform_dictionary = {}
transform_dictionary["fl_x"] = fl_x
transform_dictionary["fl_y"] = fl_y
transform_dictionary["w"] = w
transform_dictionary["h"] = h
transform_dictionary["cx"] = cx
transform_dictionary["cy"] = cy
transform_dictionary["k1"] = k1
transform_dictionary["k2"] = k2
transform_dictionary["k3"] = k3
transform_dictionary["p1"] = p1
transform_dictionary["p2"] = p2
transform_dictionary["camera_model"] = camera_model
transform_dictionary["frames"] = frames

#Save as json 
with open(os.path.join(PROCESSED_DATA_PATH, "transforms.json"), 'w') as outfile:
    json.dump(transform_dictionary, outfile)
